In [1]:
import os, pickle

In [2]:
with open("../../../training_data/8.Apos/Extra_set/features.pkl", "rb") as f:
    extras_featuresd = pickle.load(f)

len(extras_featuresd), extras_featuresd

(9,
 {'8sgj':     Residues                                                          \
           pdb label_entity_id label_asym_id label_seq_id auth_asym_id   
  0       8sgj               1             A           52            A   
  1       8sgj               1             A           53            A   
  2       8sgj               1             A           54            A   
  3       8sgj               1             A           55            A   
  4       8sgj               1             A           56            A   
  ..       ...             ...           ...          ...          ...   
  746     8sgj               1             A          941            A   
  747     8sgj               1             A          942            A   
  748     8sgj               1             A          943            A   
  749     8sgj               1             A          944            A   
  750     8sgj               1             A          945            A   
  
                       

# Make predictions

## Gather data/features

In [3]:
os.system(f"ln -s {os.getcwd().rsplit('/', 3)[0]}/training_data/utils/external/fpocket/bin/fpocket fpocket") # os.getcwd().rsplit('/', 3)[0] --> absolute path of allodb_new

256

ln: failed to create symbolic link 'fpocket': File exists


In [4]:
os.system(f"ln -s {os.getcwd().rsplit('/', 3)[0]}/training_data/utils/external/mkdssp-4.4.0-linux-x64 mkdssp-4.4.0-linux-x64") # os.getcwd().rsplit('/', 3)[0] --> absolute path of allodb_new

ln: failed to create symbolic link 'mkdssp-4.4.0-linux-x64': File exists


256

In [5]:
os.system(f"ln -s /data/fnerin/nr_database nr") # os.getcwd().rsplit('/', 3)[0] --> absolute path of allodb_new

ln: failed to create symbolic link 'nr/nr_database': File exists


256

Ran with run_psiblast.py

## Predict

Edited utils.py:
- to adjust the fpocket callable to '../fpocket'
- pocket filenames to be 1-indexed instead of 0-indexed `in_file = open(self.path + self.pdbid + '_out/pockets/pocket' + str(pockidx+1) + '_atm.pdb')`
- np.mean of neighboring PSSM etc... adjusted so that np.mean() uses axis=0 and does column-average
- function sequenceCode because in the .asn from local psiblast the sequence is hexadecimal-encoded and it needs to be decoded for the rest to work

Edited AllosESmain.py to adjust the relative location of `models.m` 

In [6]:
for pdb, feats in extras_featuresd.items():
    pdb = pdb.upper()
    chain = feats[('Residues', 'auth_asym_id')].unique().item()
    if os.path.isfile(f"{pdb}/{pdb}.asn") and not os.path.isfile(f"{pdb}/{pdb}_{chain}_result.csv"):
        os.system(f"cd {pdb} && python ../AllosES/AllosES/AllosESmain.py --PDBID {pdb} --CHAIN {chain}")

***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
******Detection pockets...******
Read successfully, PSSM matrix being extracted...
Extraction successful, exporting file now...
********************************************
******Program starts running...******
******Start extracting features...******
******Feature integration completed!******
******Start prediction...******
******Prediction completed!******
→The Pocket Rankings:
  Prediction Pocket1:	310:A,291:A,286:A,288:A,266:A,265:A,290:A,71:A,312:A,295:A,294:A,296:A,267:A,264:A,289:A,263:A,72:A,148:A,144:A,70:A,69:A,268:A
  Prediction Pocket2:	235:A,219:A,239:A,222:A,232:A,263:A,145:A,148:A,254:A,260:A,287:A,240:A,293:A,149:A,218:A,152:A,262:A,236:A,215:A,224:A,250:A,255:A,220:A
  Prediction Pocket3:	71:A,69:A,73:A,77:A,151:A,290:A,76:A,80:A,72:A,289:A
  Prediction Pocket4:	313:A,241:A,175:A,314:A,238:A,315:A,316:A,311:A,293:A,294:A,312:A,239:A,295:A
  Prediction Pocket5:	196:A,186:A,185:A,189:A,193:A,192:A,105:A,

# Process

In [7]:
import pandas as pd

In [8]:
results = {}

for pdb, feats in extras_featuresd.items():
    pdb = pdb.upper()
    chain = feats[('Residues', 'auth_asym_id')].unique().item()
    csv = f"{pdb}/{pdb}_{chain}_result.csv"
    if os.path.isfile(csv):
        results[pdb.lower()] = {
            f"pocket{pocket.iloc[0]+1}": {
                "pro_ave": pocket["pro_ave"],
                "residues": pd.DataFrame(
                    (
                        res.split(":") 
                        for res in pocket["residues"].split(",")
                    ),
                    columns=["auth_seq_id", "auth_asym_id"]
                )
            }
            for i, pocket in pd.read_csv(csv).iterrows()
        }

len(results), results

(9,
 {'8sgj': {'pocket1': {'pro_ave': 0.4566435782802614,
    'residues':    auth_seq_id auth_asym_id
    0          837            A
    1          172            A
    2          833            A
    3          830            A
    4          829            A
    5          836            A
    6          840            A
    7          210            A
    8           99            A
    9          103            A
    10         214            A
    11         244            A
    12          97            A
    13         825            A
    14         827            A
    15         826            A
    16         168            A
    17         175            A
    18         171            A
    19         102            A
    20         165            A
    21         209            A
    22         213            A
    23          98            A
    24         222            A
    25         224            A},
   'pocket54': {'pro_ave': 0.2878497901019058,
    'residues':  

In [9]:
resultsf = "alloses_results.pkl"

with open(resultsf, "wb") as f:
    pickle.dump(results, f)